In [5]:
from ipywidgets import interact,interact_manual,IntSlider
import os 
import pandas as pd
from auto_profiler import Profiler
import numpy as np
root='datasets/alg_out/'
root_ct='datasets/ct'
root_out='results'
import evalseg
evalseg.reload()
root_data = '../../datasets'
@interact
def select_dataset(dataset_name=sorted([d for d in os.listdir(root_data) if os.path.isdir(f'{root_data}/{d}')])):
    global dataset
    evalseg.reload()
    dataset = evalseg.io.NibDataset(f'{root_data}/{dataset_name}')

    @interact_manual
    def select_case(case=sorted(dataset.get_available_ids())):
        # dataset.load_all_of_case(case)
        global gto,gto_voxelsize,cto
        gto, gto_voxelsize = dataset.get_groundtruth(case)
        print(gto_voxelsize,gto.shape)
        cto, cto_voxelsize = dataset.get_CT(case)
        
        @interact
        def pred_selec(prediction=dataset.get_prediction_methods(case)):
            global pro
            pro, _ = dataset.get_prediction(prediction, case)
            # view(cto,gto,pro,gto_voxelsize,case)
                                

    


interactive(children=(Dropdown(description='dataset_name', options=('Task001_LiverTumor', 'Task002_Pancreas', …

In [10]:
view1(cto,gto,pro,gto_voxelsize,'a')

Output()

In [13]:
!conda install -c conda-forge k3d

In [3]:
def view1(ct,gt,pr,spacing,name):
    import k3d
    hasz=1
    plot = k3d.plot(grid=[0, gt.shape[0], 0, gt.shape[1], 0, gt.shape[2]], name=name, grid_auto_fit=False,camera_auto_fit=hasz)
    plot.display()
    v= k3d.voxels(ct.astype(np.uint8), opacity=0.3, compression_level=9, name='ct', group='ct',scaling=spacing)
    v.visible=1
    v.outlines=False
    plot+=v
    
    
def view(ct,gt,pr,spacing,name):
    print('working...')
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from skimage import measure
    import matplotlib.pyplot as plt
    image=ct
    
    threshold=image.min()/3 if image.min()<0 else image.min()+.1
    p = image.transpose(2,1,0)
    verts, faces, normals, values = measure.marching_cubes(p, threshold)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()
    
# view(ct,gt,pr,voxelsize,name)
